First updated @ Oct 31, 2021      Happy Halloween👻

Last updated @ Nov 2, 2021

In [8]:
# import library
import pandas as pd

## Read all movies

- get the full movie list and the imdbid

In [9]:
df = pd.read_json('http://bechdeltest.com/api/v1/getAllMovies')

In [10]:
df.tail()

,title,imdbid,id,rating,year
9368,Love Hard,10752004,10152,2,2021
9369,Cruella,,10155,3,2021
9370,West Side Story,3581652,10157,3,2021
9371,Every Time a Bell Rings,15943414,10158,3,2021
9372,Single All The Way,14315756,10161,3,2021


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9373 entries, 0 to 9372
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   9373 non-null   object
 1   imdbid  9373 non-null   object
 2   id      9373 non-null   int64 
 3   rating  9373 non-null   int64 
 4   year    9373 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 366.3+ KB


In [12]:
# save dataframe to a csv file
df.to_csv('Bechdel.csv')

## Get movie by imdbid

- then loop through the imdbid in the previous dataframe to get each movie info
- create a new dataframe and output the newer csv file

Note: The website is fragile and you may experience timeout error. Make sure to save dataframe to csv so that you don't lose everything.

In [17]:
count = 1
# create an empty dataframe
df_new = pd.DataFrame(columns=['title', 'year', 'rating', 'dubious', 'imdbid', 'id', 'submitterid', 'date', 'visible'])

# call from the latest to the earliest movie, in case the website crashes
for imdbid in df['imdbid'][-1::-1]:
    series = pd.read_json('http://bechdeltest.com/api/v1/getMovieByImdbId?imdbid=' + str(imdbid), typ='series')
    # print(series)
    df_new = df_new.append(series, ignore_index=True)
    # every 10 movies, we save to a csv file, 
    # so that we wouldn't lose everything when there is an error
    if count % 10 == 0:
        df_new.to_csv('Bechdel_detailed_reversed.csv')
    count += 1
df_new.to_csv('Bechdel_detailed_reversed.csv')

This took about 7 hours to execute

In [18]:
# reverse 
df_new = df_new.iloc[::-1].reset_index(drop=True)
df_new.to_csv('Bechdel_detailed.csv', index=False)

## Check data integrity

- check duplicates
- check row numbers
- check index

In [19]:
df = pd.read_csv('./Bechdel.csv', index_col=0)
df_to_check = pd.read_csv('./Bechdel_detailed.csv')
df_to_check

,title,year,rating,dubious,imdbid,id,submitterid,date,visible,description,status,version
0,Passage de Venus,1874.0,0.0,0.0,3155794.0,9602.0,18880.0,2021-04-02 20:58:09,1.0,NaN,NaN,NaN
1,La Rosace Magique,1877.0,0.0,0.0,14495706.0,9804.0,19145.0,2021-05-11 00:11:22,1.0,NaN,NaN,NaN
2,Sallie Gardner at a Gallop,1878.0,0.0,0.0,2221420.0,9603.0,18882.0,2021-04-03 02:25:27,1.0,NaN,NaN,NaN
3,Le singe musicien,1878.0,0.0,0.0,12592084.0,9806.0,19151.0,2021-05-11 23:38:54,1.0,NaN,NaN,NaN
4,Athlete Swinging a Pick,1881.0,0.0,0.0,7816420.0,9816.0,19162.0,2021-05-13 01:32:14,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9368,Love Hard,2021.0,2.0,0.0,10752004.0,10152.0,19735.0,2021-12-05 19:22:20,1.0,NaN,NaN,NaN
9369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Could not find movie,404.0,v1
9370,West Side Story,2021.0,3.0,0.0,3581652.0,10157.0,19743.0,2021-12-10 03:10:09,1.0,NaN,NaN,NaN
9371,Every Time a Bell Rings,2021.0,3.0,0.0,15943414.0,10158.0,19744.0,2021-12-10 08:03:02,1.0,NaN,NaN,NaN


In [20]:
def check_dataset(df_to_check):
    lengthFlag = len(df_to_check) == len(df)
    print(f'Same number of rows? {lengthFlag}, should be {len(df)} movies, got {len(df_to_check)} movies')
    if lengthFlag:
        movieFlag = True
        for i in range(len(df)):
            if df['imdbid'].iloc[i] != df_to_check['imdbid'].iloc[i]:
                print(i, df['imdbid'].iloc[i], df_to_check['imdbid'].iloc[i])
                movieFlag = False
                break
        print(f'Same movies? {movieFlag}')
    else:
        print(f'Same movies? False')

In [21]:
# check integrity and fail
check_dataset(df_to_check)

Same number of rows? True, should be 9373 movies, got 9373 movies
8374 nan nan
Same movies? False
